Імпортувати основні бібліотеки

In [ ]:
import pandas as pd
from pathlib import Path




In [50]:
path = Path("data/Кладовище.xlsx")

dfs = {}
with pd.ExcelFile(path) as xls:  # за .xlsx можна: engine="openpyxl"
    for sh in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sh, header=None, names=["number","name","dates","teritory", 'row_list'])
        df['row_list'] = sh
        if not df.empty:         # за бажанням: пропускати порожні
            dfs[sh] = df

# приклади використання:
df_all = pd.concat(dfs.values(), ignore_index=True, sort=False)
df_all = df_all.drop(columns=["number"])

df_all.head(10)


,name,dates,teritory,row_list
0,"Прізвище, ім'я",Роки народження - смерті,Територія,Ряд 1
1,Жорницька Феня Сигізмундівна,02.05.1924-13.08.1982,NaN,Ряд 1
2,Жорницький Яків Абрамович,26.08.1021 - 15.03.2006,NaN,Ряд 1
3,Шевченко Ганна Михайлівна,02.02.1912 - 15.03.1995,NaN,Ряд 1
4,Шевченко Филипп Иванович,1908-1979,NaN,Ряд 1
5,Балакирев Генадий Николаевич,15.08.1937 - 15.07.1999,NaN,Ряд 1
6,Балакирева Вера Василивна,3009.1896 - 4.7.1977,NaN,Ряд 1
7,Петрик Лідія Митрофанівна,14.10.1928 - 8.03.2012,NaN,Ряд 1
8,Куприянов Федор Васильевич,1894 - 1929,NaN,Ряд 1
9,Поліщук Марфа Павлівна,0.07.1913 - 9.11.1985,NaN,Ряд 1
